<a href="https://colab.research.google.com/github/srikg-msai22/SSLM-group9/blob/main/WebNLG_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Extraction

In [1]:
# Imports
!pip install datasets
!pip install -U pyLDAvis

import pyarrow
import re
import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import numpy as np
import nltk
nltk.download('stopwords')

from datasets import list_datasets, list_metrics, load_dataset, load_metric
from pprint import pprint
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

%matplotlib notebook
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.5 MB/s 
     |████████████████████████████████| 86 kB 3.0 MB/s 
     |████████████████████████████████| 212 kB 52.6 MB/s 
     |████████████████████████████████| 1.1 MB 53.9 MB/s 
     |████████████████████████████████| 140 kB 65.8 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 55.8 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
     |████████████████████████████████| 144 kB 17.2 MB/s 
     |████████████████████████████████| 271 kB 60.1 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
     

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

In [3]:
# Currently available datasets and metrics
datasets = list_datasets()
metrics = list_metrics()

print(f"🤩 Currently {len(datasets)} datasets are available on the hub:")
pprint(datasets, compact=True)
#print(f"🤩 Currently {len(metrics)} metrics are available on the hub:")
#pprint(metrics, compact=True)

🤩 Currently 5120 datasets are available on the hub:
['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc',
 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue',
 'ajgt_twitter_ar', 'allegro_reviews', 'allocine', 'alt', 'amazon_polarity',
 'amazon_reviews_multi', 'amazon_us_reviews', 'ambig_qa', 'americas_nli', 'ami',
 'amttl', 'anli', 'app_reviews', 'aqua_rat', 'aquamuse', 'ar_cov19',
 'ar_res_reviews', 'ar_sarcasm', 'arabic_billion_words', 'arabic_pos_dialect',
 'arabic_speech_corpus', 'arcd', 'arsentd_lev', 'art', 'arxiv_dataset',
 'ascent_kb', 'aslg_pc12', 'asnq', 'asset', 'assin', 'assin2', 'atomic',
 'autshumato', 'babi_qa', 'banking77', 'bbaw_egyptian', 'bbc_hindi_nli',
 'bc2gm_corpus', 'beans', 'best2009', 'bianet', 'bible_para', 'big_patent',
 'billsum', 'bing_coronavirus_query_set', 'biomrc', 'biosses', 'blbooks',
 'blbooksgenre', 'blended_skill_talk', 'blimp', 'blog_authorship_corpus',
 'bn_hate_speech', 'bnl_newspapers', 'bookcorpus', 'bookcorpusopen

In [4]:
# You can access various attributes of the datasets before downloading them
webnlg_data = list_datasets(with_details=True)[datasets.index('web_nlg')]

pprint(webnlg_data.__dict__)  # It's a simple python dataclass

{'author': None,
 'cardData': {'annotations_creators': ['found'],
              'configs': ['release_v1',
                          'release_v2',
                          'release_v2.1',
                          'release_v2.1_constrained',
                          'release_v2_constrained',
                          'release_v3.0_en',
                          'release_v3.0_ru',
                          'webnlg_challenge_2017'],
              'language_creators': ['crowdsourced'],
              'languages': ['en', 'ru'],
              'licenses': ['cc-by-sa-3.0', 'cc-by-nc-sa-4.0', 'gfdl-1.1'],
              'multilinguality': ['monolingual'],
              'paperswithcode_id': 'webnlg',
              'pretty_name': 'WebNLG',
              'size_categories': ['10K<n<100K'],
              'source_datasets': ['extended|other-db_pedia', 'original'],
              'task_categories': ['tabular-to-text'],
              'task_ids': ['rdf-to-text']},
 'citation': '@inproceedings{web_nlg,\n'

In [5]:
# Downloading and loading a dataset
web_nlg_train = load_dataset('web_nlg', 'release_v3.0_en', split='train')
web_nlg_dev = load_dataset('web_nlg', 'release_v3.0_en', split='dev')
web_nlg_test = load_dataset('web_nlg', 'release_v3.0_en', split='test')

Generating train split:   0%|          | 0/13211 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1667 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5713 [00:00<?, ? examples/s]

Dataset web_nlg downloaded and prepared to /root/.cache/huggingface/datasets/web_nlg/release_v3.0_en/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda. Subsequent calls will reuse this data.


Reusing dataset web_nlg (/root/.cache/huggingface/datasets/web_nlg/release_v3.0_en/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda)
Reusing dataset web_nlg (/root/.cache/huggingface/datasets/web_nlg/release_v3.0_en/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda)


## Data Sanity Checks

In [62]:
# Informations on the dataset (description, citation, size, splits, format...)
# are provided in `dataset.info` (a simple python dataclass) and also as direct attributes in the dataset object
pprint(web_nlg_train.info.__dict__)

{'builder_name': 'web_nlg',
 'citation': '@inproceedings{web_nlg,\n'
             '  author    = {Claire Gardent and\n'
             '               Anastasia Shimorina and\n'
             '               Shashi Narayan and\n'
             '               Laura Perez{-}Beltrachini},\n'
             '  editor    = {Regina Barzilay and\n'
             '               Min{-}Yen Kan},\n'
             '  title     = {Creating Training Corpora for {NLG} '
             'Micro-Planners},\n'
             '  booktitle = {Proceedings of the 55th Annual Meeting of the\n'
             '               Association for Computational Linguistics,\n'
             '               {ACL} 2017, Vancouver, Canada, July 30 - August '
             '4,\n'
             '               Volume 1: Long Papers},\n'
             '  pages     = {179--188},\n'
             '  publisher = {Association for Computational Linguistics},\n'
             '  year      = {2017},\n'
             '  url       = {https://doi.org/1

In [63]:
print(web_nlg_train)
print(web_nlg_test)
print(web_nlg_dev)

Dataset({
    features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
    num_rows: 13211
})
Dataset({
    features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
    num_rows: 5713
})
Dataset({
    features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
    num_rows: 1667
})


In [64]:
print(f"👉 Dataset len(dataset): {len(web_nlg_train)}")
print("\n👉 First item 'dataset[1]':")
pprint(web_nlg_train[0])

👉 Dataset len(dataset): 13211

👉 First item 'dataset[1]':
{'category': 'Airport',
 'dbpedia_links': [],
 'eid': 'Id1',
 'lex': {'comment': ['good', 'good'],
         'lang': ['', ''],
         'lid': ['Id1', 'Id2'],
         'text': ['The Aarhus is the airport of Aarhus, Denmark.',
                  'Aarhus Airport serves the city of Aarhus, Denmark.']},
 'links': [],
 'modified_triple_sets': {'mtriple_set': [['Aarhus_Airport | cityServed | '
                                           '"Aarhus, Denmark"']]},
 'original_triple_sets': {'otriple_set': [['Aarhus_Airport | cityServed | '
                                           '"Aarhus, Denmark"@en']]},
 'shape': '(X (X))',
 'shape_type': 'NA',
 'size': 1,
 'test_category': ''}


## EDA/Stats of the datasets

### String Length Stats

In [60]:
import numpy as np

lengths = []
for i in range(len(web_nlg_train)):
  lengths.append(web_nlg_train[i]['lex']['text'][0].count(' ') + 1)

print(np.max(lengths))
print(np.min(lengths))
print(np.mean(lengths))
print(np.median(lengths))
print(np.percentile(lengths, 25))
print(np.percentile(lengths, 75))

72
3
19.662478237832108
18.0
11.0
26.0


In [103]:
lengths_test = []
max_idx_test = 0
for i in range(len(web_nlg_test)):
  if len(web_nlg_test[i]['lex']['text']) > 0:
    lengths_test.append(web_nlg_test[i]['lex']['text'][0].count(' ') + 1)
    if (web_nlg_test[max_idx_test]['lex']['text'][0].count(' ') + 1) < lengths_test[-1]:
      max_idx_test = i


print(np.max(lengths_test))
print(np.min(lengths_test))
print(np.mean(lengths_test))
print(np.median(lengths_test))
print(np.percentile(lengths_test, 25))
print(np.percentile(lengths_test, 75))
print(max_idx_test)

80
3
21.81825114387392
20.0
13.0
29.0
1492


In [100]:
lengths_dev = []
max_idx = 0
for i in range(len(web_nlg_dev)):
  lengths_dev.append(web_nlg_dev[i]['lex']['text'][0].count(' ') + 1)
  if (web_nlg_dev[max_idx]['lex']['text'][0].count(' ') + 1) < lengths_dev[-1]:
    max_idx = i

print(np.max(lengths_dev))
print(np.min(lengths_dev))
print(np.mean(lengths_dev))
print(np.median(lengths_dev))
print(np.percentile(lengths_dev, 25))
print(np.percentile(lengths_dev, 75))
print(max_idx)

60
4
19.69226154769046
18.0
11.0
26.0
1659


In [40]:
index_min = min(range(len(lengths)), key=lengths.__getitem__)
index_max = max(range(len(lengths)), key=lengths.__getitem__)

print(web_nlg_train[index_min]['lex']['text'][0])
print(web_nlg_train[index_max]['lex']['text'][0])

print(web_nlg_train[index_min]['modified_triple_sets']['mtriple_set'][0]) # Min length having original text
print(web_nlg_train[index_max]['modified_triple_sets']['mtriple_set'][0]) # Max length having original text
print(len(web_nlg_train[index_max]['modified_triple_sets']['mtriple_set'][0])) # Number of predicates from the max length having original text

Hypermarcas produces cosmetics.
The leader of Azerbaijan is called the Prime Minister of Azerbaijan. The capital of Azerbaijan is Baku. In Baku is the Baku Turkish Martyrs's Memorial located which was designed by Huseyin Butuner and Hilmi Guner. Turk Sehitleri Aniti is the native name for the Baku Turkish Martyrs' Memorial, which is dedicated to the Ottoman Army soldiers killed in the Battle of Baku. The source of legislature in Azerbaijan is the National Assembly.
['Hypermarcas | product | Cosmetics']
['Azerbaijan | capital | Baku', 'Azerbaijan | leaderTitle | Prime_Minister_of_Azerbaijan', 'Baku_Turkish_Martyrs\'_Memorial | dedicatedTo | "Ottoman Army soldiers killed in the Battle of Baku"@en', "Baku_Turkish_Martyrs'_Memorial | location | Azerbaijan", 'Baku_Turkish_Martyrs\'_Memorial | nativeName | "Türk Şehitleri Anıtı"@en', 'Baku_Turkish_Martyrs\'_Memorial | designer | "Hüseyin Bütüner and Hilmi Güner"@en', 'Azerbaijan | legislature | National_Assembly_(Azerbaijan)']
7


In [101]:
print(web_nlg_dev[1659]['lex']['text'][0])
print(web_nlg_dev[1659]['modified_triple_sets']['mtriple_set'][0]) # Max length having original text
print(len(web_nlg_dev[1659]['modified_triple_sets']['mtriple_set'][0])) # Number of predicates from the max length having original text

The National Assembly is the official legislature of Azerbaijan which is led by a Prime Minister. The capital city is Baku which is the location of the Turkish Martyrs' memorial designed in red granite and white marble by Huseyin Butuner and Hilmi Guner. The memorial is dedicated to the Ottoman army soldiers who were killed in the Battle of Baku.
['Azerbaijan | capital | Baku', 'Baku_Turkish_Martyrs\'_Memorial | material | "Red granite and white marble"', 'Azerbaijan | leaderTitle | Prime_Minister_of_Azerbaijan', 'Baku_Turkish_Martyrs\'_Memorial | dedicatedTo | "Ottoman Army soldiers killed in the Battle of Baku"', "Baku_Turkish_Martyrs'_Memorial | location | Azerbaijan", 'Baku_Turkish_Martyrs\'_Memorial | designer | "Hüseyin Bütüner and Hilmi Güner"', 'Azerbaijan | legislature | National_Assembly_(Azerbaijan)']
7


In [104]:
print(web_nlg_test[1492]['lex']['text'][0])
print(web_nlg_test[1492]['modified_triple_sets']['mtriple_set'][0]) # Max length having original text
print(len(web_nlg_test[1492]['modified_triple_sets']['mtriple_set'][0])) # Number of predicates from the max length having original text

The Acharya Institute of Technology is in Bangalore (which was founded by Kempe Gowda I); the address of the campus is as follows: Soldevanahalli, Acharya Dr. Sarvapalli Radhakrishnan Road, Hessarghatta Main Road, Bangalore - 560090. The All India Council for Technical Education, which is located in Mumbai, gave the Acharya Institute of Technology the status of 'Technical Campus'. There are around 700 postgraduate students at the Acharya Institute of Technology, lead by Dr. G.P. Prabhukumar, who is the institute's director.
['Acharya_Institute_of_Technology | campus | "In Soldevanahalli, Acharya Dr. Sarvapalli Radhakrishnan Road, Hessarghatta Main Road, Bangalore – 560090."', 'All_India_Council_for_Technical_Education | location | Mumbai', 'Acharya_Institute_of_Technology | director | "Dr. G. P. Prabhukumar"', 'Bangalore | founder | Kempe_Gowda_I', 'Acharya_Institute_of_Technology | city | Bangalore', 'Acharya_Institute_of_Technology | wasGivenTheTechnicalCampusStatusBy | All_India_Cou

In [61]:
pred_lengths = []
for i in range(len(web_nlg_train)):
  pred_lengths.append(len(web_nlg_train[i]['modified_triple_sets']['mtriple_set'][0]))

print(np.max(pred_lengths))
print(np.min(pred_lengths))
print(np.mean(pred_lengths))
print(np.median(pred_lengths))
print(np.percentile(pred_lengths, 25))
print(np.percentile(pred_lengths, 75))

7
1
2.906592990689577
3.0
2.0
4.0


In [87]:
pred_lengths_test = []
for i in range(len(web_nlg_test)):
  pred_lengths_test.append(len(web_nlg_test[i]['modified_triple_sets']['mtriple_set'][0]))

print(np.max(pred_lengths_test))
print(np.min(pred_lengths_test))
print(np.mean(pred_lengths_test))
print(np.median(pred_lengths_test))
print(np.percentile(pred_lengths_test, 25))
print(np.percentile(pred_lengths_test, 75))

7
1
3.1897426921057237
3.0
2.0
4.0


In [88]:
pred_lengths_dev = []
for i in range(len(web_nlg_dev)):
  pred_lengths_dev.append(len(web_nlg_dev[i]['modified_triple_sets']['mtriple_set'][0]))

print(np.max(pred_lengths_dev))
print(np.min(pred_lengths_dev))
print(np.mean(pred_lengths_dev))
print(np.median(pred_lengths_dev))
print(np.percentile(pred_lengths_dev, 25))
print(np.percentile(pred_lengths_dev, 75))

7
1
2.904019196160768
3.0
2.0
4.0


#### String Length Stats in a glance

Explanation | Stat
------------|------
Maximum number of tokens in a string | 72
Minimum number of tokens in a string | 3
Mean number of tokens in a string | 19.66
Median number of tokens in a string | 18.00
Number of tokens in the 1st Quantile in a string | 11.00
Number of tokens in the 3rd Quantile in a string | 26.00
-------|------
Number of predicates for the max string | 7
-------|------
Maximum number of predicates for a given string | 7
Minimum number of predicates for a given string | 1
Mean number of predicates for a given string | 2.9
Median number of predicates for a given string | 3.0
Number of predicates in the 1st Quantile for a string | 2.0
Number of predicates in the 3rd Quantile for a string | 4.0

### Predicate/Properties Repitition Stats

In [54]:
predicates = []
temp_lens = []
for i in range(len(web_nlg_train)):
  predicate = web_nlg_train[i]['modified_triple_sets']['mtriple_set'][0]
  temp_lens.append(len(web_nlg_train[i]['modified_triple_sets']['mtriple_set']))
  
  if len(predicate) == 1:
    predicates.append(predicate[0].split(' | ')[1])
  else:
    for j in range(len(predicate)):
      predicates.append(predicate[j].split(' | ')[1])

print(len(predicates))
print(predicates)
print(len(list(set(predicates))))
print(len(list(set(temp_lens))))

38399
['cityServed', 'cityServed', 'elevationAboveTheSeaLevel', 'location', 'operatingOrganisation', 'operatingOrganisation', 'runwayLength', 'runwayLength', 'runwayName', 'runwayName', 'country', 'isPartOf', 'isPartOf', 'isPartOf', '1stRunwayLengthFeet', '1stRunwaySurfaceType', '3rdRunwayLengthFeet', 'icaoLocationIdentifier', 'elevationAboveTheSeaLevel', 'locationIdentifier', 'runwayLength', 'runwayLength', 'runwayLength', 'runwayName', 'runwayName', '1stRunwayLengthFeet', 'cityServed', 'cityServed', 'locationIdentifier', 'runwayLength', 'runwayLength', 'elevationAboveTheSeaLevel', 'location', 'location', 'location', 'operatingOrganisation', 'runwayLength', 'runwayLength', 'runwayLength', 'runwayName', 'elevationAboveTheSeaLevelInFeet', 'elevationAboveTheSeaLevel', 'location', 'runwayLength', 'runwayLength', 'runwayName', 'runwayName', 'iataLocationIdentifier', 'icaoLocationIdentifier', 'elevationAboveTheSeaLevel', 'location', 'location', 'nativeName', 'operatingOrganisation', 'operat

In [55]:
predicates_test = []

for i in range(len(web_nlg_test)):
  predicate = web_nlg_test[i]['modified_triple_sets']['mtriple_set'][0]
  
  if len(predicate) == 1:
    predicates_test.append(predicate[0].split(' | ')[1])
  else:
    for j in range(len(predicate)):
      predicates_test.append(predicate[j].split(' | ')[1])

print(len(predicates_test))
print(predicates_test)
print(len(list(set(predicates_test))))

18223
['location', 'league', 'country', 'nickname', 'ground', 'birthDate', 'occupation', 'city', 'height', 'ground', 'club', 'populationMetro', 'leaderTitle', 'type', 'populationDensity', 'governmentType', 'precededBy', 'followedBy', 'populationDensity', 'leaderTitle', 'country', 'elevationAboveTheSeaLevel', 'timeZone', 'deathDate', 'foundingDate', 'deathPlace', 'areaTotal', 'populationDensity', 'birthDate', 'areaCode', 'hasToItsWest', 'established', 'location', 'municipality', 'category', 'hasToItsNorth', 'officialLanguage', 'operatingOrganisation', 'location', 'elevationAboveTheSeaLevel', 'foundingDate', 'product', 'location', 'areaTotal', 'location', 'industry', 'genre', 'precededBy', 'precededBy', 'campus', 'director', 'affiliation', 'country', 'deathPlace', 'birthPlace', 'ethnicGroup', 'artist', 'producer', 'runtime', 'recordedIn', 'recordedIn', 'genre', 'recordLabel', 'publisher', 'literaryGenre', 'deathPlace', 'birthPlace', 'award', 'followedBy', 'producer', 'album', 'publisher'

In [56]:
predicates_dev = []

for i in range(len(web_nlg_dev)):
  predicate = web_nlg_dev[i]['modified_triple_sets']['mtriple_set'][0]
  
  if len(predicate) == 1:
    predicates_dev.append(predicate[0].split(' | ')[1])
  else:
    for j in range(len(predicate)):
      predicates_dev.append(predicate[j].split(' | ')[1])

print(len(predicates_dev))
print(predicates_dev)
print(len(list(set(predicates_dev))))

4841
['leader', 'runwayLength', 'elevationAboveTheSeaLevel', 'location', 'location', 'runwayName', 'runwayName', 'operatingOrganisation', 'location', 'elevationAboveTheSeaLevel', 'cityServed', 'runwayName', 'runwayLength', 'runwayName', 'cityServed', 'countySeat', 'cityServed', 'runwayName', 'runwayName', 'cityServed', 'location', 'mayor', 'location', 'language', 'leader', 'capital', 'leader', 'leader', 'language', 'leader', 'isPartOf', 'leader', 'isPartOf', 'headquarter', 'country', 'isPartOf', 'chief', 'activeYearsStartYear', 'associatedBand/associatedMusicalArtist', 'genre', 'genre', 'origin', 'associatedBand/associatedMusicalArtist', 'background', 'birthDate', 'recordLabel', 'recordLabel', 'associatedBand/associatedMusicalArtist', 'birthPlace', 'associatedBand/associatedMusicalArtist', 'background', 'genre', 'deathPlace', 'associatedBand/associatedMusicalArtist', 'genre', 'recordLabel', 'activeYearsStartYear', 'genre', 'recordLabel', 'recordLabel', 'associatedBand/associatedMusical

In [58]:
train_preds_set = set(predicates)
test_intersection = train_preds_set.intersection(predicates_test)
test_intersection_list = list(test_intersection)

print(len(list(set(predicates_test))))
print(len(test_intersection_list))

dev_intersection = train_preds_set.intersection(predicates_dev)
dev_intersection_list = list(dev_intersection)

print(len(list(set(predicates_dev))))
print(len(dev_intersection_list))

227
187
290
290


#### Predicate/Property Repition Stats in a glance

Explanation                                | Stat
-------------------------------------------|------------------
Number of Train Predictes                  | 372
Number of Dev Predictes                    | 290
Number of Dev Predictes Common with Train  | 290
Number of Test Predictes                   | 227
Number of Test Predictes Common with Train | 187
Number of Test Predictes Unique to Test    | 40